In [11]:
from typing import List

import torch
from transformers import AutoModelForCausalLM
from src.data_tokenizer import RapDataTokenizer
from src.lyrics_cleaner import clean_generated_lyrics
# используем тот же GPU, что и для обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# путь, куда была сохранена модель после обучения
MODEL_PATH = "./final_model"
MODEL_NAME = "sberbank-ai/rugpt3small_based_on_gpt2"

# загружаем токенизатор и модель
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH).to(device)
rap_tokenizer = RapDataTokenizer(MODEL_NAME)

print(f" Модель и токенизатор готовы на устройстве: {device}")

Добавлено 11 специальных токенов
Размер словаря: 50268
 Модель и токенизатор готовы на устройстве: cpu


In [53]:
def generate_and_clean_rap_lyrics(
    prompt: str,
    max_tokens: int = 100,
    temperature: float = 0.85,
    top_k: int = 40,
    top_p: float = 0.85,
    repetition_penalty: float = 1.8
) -> List[str]:
    """
    Создает лирику на основе промпта, генерирует текст и очищает его от токенов.
    Использует глобальные объекты 'model', 'rap_tokenizer', и 'device'.
    """

    # 1. Подготовка ввода
    inputs = rap_tokenizer.tokenizer(
        prompt,
        return_tensors='pt',
        padding=True,
        truncation=True
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Максимальная длина генерации (длина промпта + желаемое число новых токенов)
    max_length = input_ids.shape[1] + max_tokens

    # 2. Генерация
    model.eval()
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            do_sample=True,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            num_return_sequences=1,
            pad_token_id=rap_tokenizer.tokenizer.pad_token_id
        )

    # 3. Декодирование сырого вывода
    raw_output = rap_tokenizer.tokenizer.decode(output_ids[0], skip_special_tokens=False)

    # 4. Очистка и форматирование
    cleaned_lyrics = clean_generated_lyrics(raw_output)

    return cleaned_lyrics

In [54]:
test_depo = (
    f"<META> [Boulevard Depo] [Dark, Lyrical, Philosophical, Abstract, Conceptual, Cloud Rap] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Мой флоу течет сквозь стены, будто плазма,\n"
    f"Нам осталась только пыль из мира кармы.\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>" # Просим модель продолжить припевом
)

print("--- ГЕНЕРАЦИЯ DEPO ---")
lyrics_kai = generate_and_clean_rap_lyrics(prompt=test_depo, repetition_penalty=1.9, temperature=1.1)
print('\n'.join(lyrics_kai))

--- ГЕНЕРАЦИЯ DEPO ---
[Verse]

Мой флоу течет сквозь стены, будто плазма,
Нам осталась только пыль из мира кармы.
[Chorus]
* В голове вакуумная волна А а Меня тянет вверх и выше На шее плети сакуры Но я не чувствую боль Я на грани понимания Где моя суть?, где моё место?
Что со мной случится завтра Если меня кто то обидит И тогда мы будем искать виноватых Каждый день для нас был бы последним По другому быть так же сложно Сожалею Мы живём во лжи но ведь люди меняются Не могу понять людей Ведь мир меняется лишь по тем причинам Ощутить вкус тех ощущений Как


In [69]:
prompt_kai_abstract = (
    f"<META> [Kai Angel] [Industrial, Dark, Conceptual] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Зачем смотреть вперед, если там только ноль?\n"
    f"Я заблокировал вход в свой персональный мозг, это контроль.\n"
    f"Моя машина — это танк, и он едет мимо всех.\n"
)


print("--- ГЕНЕРАЦИЯ KAI ANGEL ABSTRACT ---")
lyrics_kai = generate_and_clean_rap_lyrics(prompt=prompt_kai_abstract, repetition_penalty=1.9, temperature=1.4)
print('\n'.join(lyrics_kai))

--- ГЕНЕРАЦИЯ KAI ANGEL ABSTRACT ---
[Verse]

Зачем смотреть вперед, если там только ноль?
Я заблокировал вход в свой персональный мозг, это контроль.
Моя машина — это танк, и он едет мимо всех.
попытках попасть внутрь сознания Я закрываю глаза чтобы видеть себя с разных ракурсов Теперь я уже сам такой же как весь день Вокруг тебя все враги Мои стены теперь выглядят будто ты знаешь всё Знаю наизусть Если что нибудь будет со мной потом мы не услышим ни слова Нет И мне наплевать на всю эту суку до последнего патрона Не нужен мой звук они знают меня лучше других Они скажут: Посмотри под потолок Ты всегда был голодным Мне хочется стать другим для неё Так нужно Это точно так Как хочешь


In [61]:
prompt_kai_chorus = (
    f"<META> [Kai Angel] [Industrial, Cloud Rap, New School] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Красный код на студии, красный свет в моих глазах.\n"
    f"Я не сплю, я строю мир на твоих костях.\n"
    f"Они думали, что знают, но не знали ничего.\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)
print("--- ГЕНЕРАЦИЯ KAI ANGEL CHORUS ---")
lyrics_kai = generate_and_clean_rap_lyrics(prompt=prompt_kai_chorus, repetition_penalty=2.1, temperature=1.9)
print('\n'.join(lyrics_kai))

--- ГЕНЕРАЦИЯ KAI ANGEL CHORUS ---
[Verse]

Красный код на студии, красный свет в моих глазах.
Я не сплю, я строю мир на твоих костях.
Они думали, что знают, но не знали ничего.
[Chorus]

Мой день начинается утром Как обычно мой час с того времени Не забывай моё имя И ни о ком кроме тебя Даже будь он проклят Ха Сними очки и надень это пальто Только так Я люблю смотреть как мы лежим под водой О чём сейчас мечтаем Ты уже взрослая девушка Знаю точно!
Учись ценить жизнь Это всё твоё Изумительный запах моего тела Так прекрасно видеть твои слёзы Твои глаза Счастливые капли моей души Хочу побыть там навсегда Эти бриллианты говорят мне больше Чем все мои


In [70]:
prompt_pharaoh = (
    f"<META> [Pharaoh] [Emo Trap, Dark, Melancholic, Cloud Rap] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Мне так одиноко, я сжёг все мосты, о да\n"
    f"Ты даже не знаешь, сколько лет мне теперь\n"
    f"Я мог бы тебя проводить в темноту, но не в дверь\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)
print("\n--- PHARAOH (ДИКО, НАПРИМЕР) ---")
lyrics_pharaoh = generate_and_clean_rap_lyrics(
    prompt=prompt_pharaoh,
    temperature=1.0,
    repetition_penalty=1.5
)
print('\n'.join(lyrics_pharaoh))


--- ТЕСТ 2: PHARAOH (ДИКО, НАПРИМЕР) ---
[Verse]

Мне так одиноко, я сжёг все мосты, о да
Ты даже не знаешь, сколько лет мне теперь
Я мог бы тебя проводить в темноту, но не в дверь

[Chorus]

И вот что ты скажешь?
Я никогда к тебе больше ничего и близко Не буду скрывать свою любовь Нет нет И меня зовут только твой голос Ведь на моих глазах слёзы Но с тобой рядом другой лишь по другому зову Что ж Теперь твоя очередь быть моей женой Или же всё то время проведённое вместе Мы будем жить без лишних чувств Ты слышишь моё сердце будто песня В горле пересохло Сжги их как можно скорее Чтобы увидеть снова ту самую Любовь Как давно было нам обоим Так долго мы хотели забыть Всё это


In [71]:
prompt_saluki = (
    f"<META> [Saluki] [Trap, New School, Smooth, Lyrical] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Я выкурил все огни в этих окнах и вышел\n"
    f"Теперь мой бренд — это знак, он дороже всех слов\n"
    f"Мне нужна только ты, но тебя слишком много вокруг\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)
print("\n--- SALUKI (ВЕНЕРА) ---")
lyrics_saluki = generate_and_clean_rap_lyrics(
    prompt=prompt_saluki,
    temperature=0.7,
    repetition_penalty=1.3
)
print('\n'.join(lyrics_saluki))


--- SALUKI (ВЕНЕРА) ---
[Verse]

Я выкурил все огни в этих окнах и вышел
Теперь мой бренд — это знак, он дороже всех слов
Мне нужна только ты, но тебя слишком много вокруг

[Chorus]

И я не знаю как быть с этим дерьмом Эй Я курю на балконе у окна А а Мы курим так же мы пьём вино В комнате пахнет хламом Е Это дым из моей головы е У меня есть всё для того чтоб вы сдохли Но теперь мне нужно больше воздуха Они хотят мою жизнь забрать их Деньги Мне нужны деньги, чтобы купить себе новый бэйби Кэш Сука хочет моё тело, хочу её запах Она даёт мне свой флоу Мой парень говорит что она любит


In [74]:
prompt_husky = (
    f"<META> [Husky] [Experimental, Aggressive, Social, Dark] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Нам остался только бетон и холодный свет из окна\n"
    f"Здесь всё одно: и смерть, и жизнь, и моя голова\n"
    f"Каждый день, как копия, каждый шаг, как ловушка\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)
print("\nХАСКИ (ПАНЕЛЬКА) ---")
lyrics_husky = generate_and_clean_rap_lyrics(
    prompt=prompt_husky,
    temperature=0.55,
    repetition_penalty=1.6
)
print('\n'.join(lyrics_husky))


ХАСКИ (ПАНЕЛЬКА) ---
[Verse]

Нам остался только бетон и холодный свет из окна
Здесь всё одно: и смерть, и жизнь, и моя голова
Каждый день, как копия, каждый шаг, как ловушка

[Chorus]

Я не хочу быть собой Я я Не буду никем другим Ты будешь мною гордиться И все мои чувства к тебе остынут А а Мы будем жить так долго Но мы снова встретимся вновь в аду Когда ты уйдёшь навсегда Все твои мечты сбудутся Так много боли вокруг Тебя нет больше никого На свете столько всего другого В этой жизни нас всех ждут лишь одни несчастья Е е Она будет ждать меня на могиле с цветами О о Как же мне тебя полюбить?
Ведь твоя любовь для них слишком скучна Слишком Ску


In [75]:
prompt_dark_prince_true = (
    f"<META> [Тёмный Принц] [Trap, Aggressive, Scammer, New School] </META>"
    f"\n<LYRICS>"
    f"\n<SEC><VERSE>\n"
    f"Моя цель — её разрушить.\n"
    f"Факаю тебя в маске, что с улыбкою по уши.\n"
    f"Я воркаю один, за минуту три лога, да, мои пальцы в крови.\n"
    f"Мне нужен профит, нужен обход Vinted.\n"
    f"</SEC>"
    f"\n<SEC><CHORUS>"
)

# ----------------------------------------------------
# Запуск теста с новым, точным контекстом
# ----------------------------------------------------
print("--- ТЕСТ: ТЁМНЫЙ ПРИНЦ (SCAMMER TRAP) ---")
lyrics_pharaoh_papa_true = generate_and_clean_rap_lyrics(
    prompt=prompt_dark_prince_true,
    temperature=0.9,
    repetition_penalty=1.8
)
print('\n'.join(lyrics_pharaoh_papa_true))

--- ТЕСТ: ТЁМНЫЙ ПРИНЦ (SCAMMER TRAP) ---
[Verse]

Моя цель — её разрушить.
Факаю тебя в маске, что с улыбкою по уши.
Я воркаю один, за минуту три лога, да, мои пальцы в крови.
Мне нужен профит, нужен обход Vinted.
[Chorus]
 (Воу) Мой план не для всех и я всегда буду рядом Я на своём поле со своим парнем это мой косяк Но он мне платит как надо И твой друг должен быть самым лучшим Ты хочешь меня обмануть?
Нет нет Ну а если ты думаешь то напрасно, тогда слушай: Убей себя сам или подари ему свой шарм Или сделай его самой прекрасной из моих подруг Эй!
Забери всё своё тело прямо сейчас А теперь смотри сюда Давись больше так сильно Ведь твоё сердце разорвёт этот
